<a href="https://colab.research.google.com/github/gRedDeer/kaggle_notebooks/blob/main/Loan_Approval_%7C_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s4e10_path = kagglehub.competition_download('playground-series-s4e10')
itshappy_ps4e9_original_data_loan_approval_prediction_path = kagglehub.dataset_download('itshappy/ps4e9-original-data-loan-approval-prediction')

print('Data source import complete.')


# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv', index_col='id')
original = pd.read_csv('/kaggle/input/ps4e9-original-data-loan-approval-prediction/credit_risk_dataset.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv',index_col='idca')
train.head()
#test.head()

In [ ]:
train.info()

In [ ]:
train.describe()

# EDA
#### Competition vs. Original
First, let's see if orignal and competition data differ in distribution.

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
f.set_figheight(5)
f.set_figwidth(12)

colors = ['green', 'grey']
ax1.pie(train.loan_status.value_counts(), labels=train.loan_status.value_counts().index, autopct="%.0f%%", colors=colors)
ax1.set_title('Competition Target distribution')

ax2.pie(original.loan_status.value_counts(), labels=original.loan_status.value_counts().index, autopct="%.0f%%", colors=colors)
ax2.set_title('Original Target distribution')

plt.show()

#### Adversarial validation

I take the code for adversarial validation from the Kaggle Book: https://github.com/PacktPublishing/The-Kaggle-Book/blob/main/chapter_06/adversarial-validation-example.ipynb

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")

X =  pd.concat([train, original], axis=0)
y = [0] * len(train) + [1] * len(original)

X['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}, inplace=True)
X['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}, inplace=True)
X['loan_grade'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}, inplace=True)
X['cb_person_default_on_file'].replace({'N': 0, 'Y': 1}, inplace=True)
X['person_emp_length'] = X['person_emp_length'].fillna(original['person_emp_length'].mean())
X['person_emp_length'] = X['person_emp_length'].astype(int)
X['loan_int_rate'] = X['loan_int_rate'].fillna(original['loan_int_rate'].mean())
X['loan_int_rate'] = (X['loan_int_rate'] * 100).astype(int)
X['loan_percent_income'] = (X['loan_percent_income'] * 100).astype(int)

model = RandomForestClassifier(random_state=0)
cv_preds = cross_val_predict(model, X, y, cv=5, n_jobs=-1, method='predict_proba')

score = roc_auc_score(y_true=y, y_score=cv_preds[:,1])
print(f"roc-auc score: {score:0.3f}")

Both Target distibution and adversarial validation (roc-auc not around ~0.5) show difference in distributions. Hence, it's doubtful, whether original data should be added.

#### Train, Test feature distribution comparison

In [ ]:
num_cols = ['person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
def plot_distribution_pairs(train, test, feature, hue="set", palette=None):
    data_df = train.copy()
    data_df['set'] = 'train'
    data_df = pd.concat([data_df, test.copy()]).fillna('test')

    f, axes = plt.subplots(1, 2, figsize=(14, 6))
    for i, s in enumerate(data_df[hue].unique()):
        selection = data_df.loc[data_df[hue]==s, feature]
        # Filter 'selection' to include only the central 95% of the data
        q_025, q_975 = np.percentile(selection, [2.5, 97.5])
        selection_filtered = selection[(selection >= q_025) & (selection <= q_975)]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            sns.histplot(selection_filtered, color=palette[i], ax=axes[0], label=s)
            sns.boxplot(x=hue, y=feature, data=data_df, palette=palette, ax=axes[1])
    axes[0].set_title(f"Paired train/test distributions of {feature}")
    axes[1].set_title(f"Paired train/test boxplots of {feature}")
    axes[0].legend()
    axes[1].legend()
    plt.show()

color_list = ["#A5D7E8", "#576CBC", "#19376D", "#0B2447"]
for feature in num_cols:
    plot_distribution_pairs(train, test, feature, palette=color_list)

In [ ]:
X =  pd.concat([train.drop('loan_status', axis=1), test], axis=0)
y = [0] * len(train) + [1] * len(test)

X['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}, inplace=True)
X['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}, inplace=True)
X['loan_grade'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}, inplace=True)
X['cb_person_default_on_file'].replace({'N': 0, 'Y': 1}, inplace=True)
X['person_emp_length'] = X['person_emp_length'].astype(int)
X['loan_int_rate'] = (X['loan_int_rate'] * 100).astype(int)
X['loan_percent_income'] = (X['loan_percent_income'] * 100).astype(int)

model = RandomForestClassifier(random_state=0)
cv_preds = cross_val_predict(model, X, y, cv=5, n_jobs=-1, method='predict_proba')

score = roc_auc_score(y_true=y, y_score=cv_preds[:,1])
print(f"roc-auc score: {score:0.3f}")

Conclusion: train and test distributions are very close, which is proved both by visualisations and adversarial validation.

# Data preprocessing

In [ ]:
X = pd.concat([train, original], ignore_index=True)
X = X.fillna('None')
def preprocess(df):
    df['loan_to_income'] = ((df['loan_amnt'] / df['person_income'])).astype('string').astype('category')
    df['age_income_interaction'] = (df['person_age'] * df['person_income']).astype('string').astype('category')
    df['loan_to_emp_length_ratio'] = (df['loan_amnt'] / df['person_emp_length'].replace({'None': original['person_emp_length'].mean()}).astype('float')).astype('string').astype('category')
    monthly_income = df['person_income'] / 12
    df['monthly_debt'] = (df['loan_amnt'] * (1 + df['loan_int_rate'].replace({'None': original['loan_int_rate'].mean()}).astype(float)) / 12)
    df['dti_ratio'] = (df['monthly_debt'] / monthly_income).astype('string').astype('category')
    df['monthly_debt'] = df['monthly_debt'].astype('string').astype('category')
    df['risk_flag'] = (np.where((df['cb_person_default_on_file'] == 'Y') & (df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0))
    df['risk_flag'] = df['risk_flag'].astype('category')
    df['person_home_ownership'] = df['person_home_ownership'].astype('category')
    df['loan_intent'] = df['loan_intent'].astype('category')
    df['loan_grade'] = df['loan_grade'].astype('category')
    df['cb_person_default_on_file'] = df['cb_person_default_on_file'].astype('category')
    df['person_emp_length'] = df['person_emp_length'].astype('string').astype('category')
    df['loan_int_rate'] = (df['loan_int_rate'].replace({'None': original['loan_int_rate'].mean()}).astype(float) * 100).astype('string').astype('category')
    df['loan_percent_income'] = (df['loan_percent_income'].astype(float) * 100).astype('string').astype('category')


preprocess(test)
preprocess(X)
y = X.pop('loan_status')

X.head()
# y = df['loan_status']
# X = df.drop('loan_status', axis=1)

# Model

In [ ]:
import optuna
from optuna.samplers import TPESampler

# Using optuna to tune hyperparameters a little bit
def objective(trial):
    params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
    'iterations': 1000,
    'depth': trial.suggest_int('depth', 6, 12),
    'random_strength':0,
    'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-1, 1.0, log=True),
    'task_type':'GPU',
    'random_seed':42,
    'verbose':False
    }

    model = CatBoostClassifier(**params)
    cv = StratifiedKFold(5, shuffle=True, random_state=0)
    cv_splits = cv.split(X, y)
    scores = list()
    for train_idx, val_idx in cv_splits:
        X_train_fold, X_val_fold = X.loc[train_idx], X.loc[val_idx]
        y_train_fold, y_val_fold = y.loc[train_idx], y.loc[val_idx]
        X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X.columns.values)
        X_valid_pool = Pool(X_val_fold, y_val_fold, cat_features=X.columns.values)
        model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=False, early_stopping_rounds=200)
        val_pred = model.predict_proba(X_valid_pool)[:, 1]
        score = roc_auc_score(y_val_fold, val_pred)
        scores.append(score)
    return np.mean(scores)

sqlite_db = "sqlite:///catboost.db"
study_name = "catboost"
optimize = False
if optimize:
    study = optuna.create_study(storage=sqlite_db, study_name=study_name,
                                sampler=TPESampler(n_startup_trials=35, multivariate=True, seed=0),
                                direction="maximize", load_if_exists=True)

    study.optimize(objective, n_trials=100)
    print(f"best optimized roc-auc: {study.best_value:0.5f}") # 0.96664
    print(f"best hyperparameters: {study.best_params}")
    catboost_params = study.best_params
else:
    catboost_params = {
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'learning_rate': 0.08114394459649094,
        'iterations': 1000,
        'depth': 6,
        'random_strength':0,
        'l2_leaf_reg': 0.7047064221215757,
        'task_type':'GPU',
        'random_seed':42,
        'verbose':False
    }

The TPESampler is set to first try random optimization for the first 35 trials and then perform a TPE Bayesian Search. I also use the multivariate approach, which is reported to outperform the independent TPE.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from catboost import Pool, CatBoostClassifier

cv = StratifiedKFold(5, shuffle=True, random_state=0)
cv_splits = cv.split(X, y)
scores = []
test_preds = []
X_test_pool = Pool(test, cat_features=X.columns.values)
for i, (train_idx, val_idx) in enumerate(cv_splits):
    model = CatBoostClassifier(**catboost_params)
    X_train_fold, X_val_fold = X.loc[train_idx], X.loc[val_idx]
    y_train_fold, y_val_fold = y.loc[train_idx], y.loc[val_idx]
    X_train_pool = Pool(X_train_fold, y_train_fold, cat_features=X.columns.values)
    X_valid_pool = Pool(X_val_fold, y_val_fold, cat_features=X.columns.values)
    model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=False, early_stopping_rounds=200)
    val_pred = model.predict_proba(X_valid_pool)[:, 1]
    score = roc_auc_score(y_val_fold, val_pred)
    scores.append(score)
    test_pred = model.predict_proba(X_test_pool)[:, 1]
    test_preds.append(test_pred)
    print(f'Fold {i + 1} roc_auc_score: {score}')
print(f'Cross-validated roc_auc_score: {np.mean(scores):.3f} +/- {np.std(scores):.3f}')
print(f'Max roc_auc_score score: {np.max(scores):.3f}')
print(f'Min roc_auc_score score: {np.min(scores):.3f}')

# Explainability
To understand how the model actually works, the Catboost model itself is used to calculate SHAP (SHapley Additive exPlanations) values for each feature in the training data. The SHAP values are obtained by using **shap.TreeExplainer**. Then we create a summary plot visualizing the impact of different features on the model predictions and a violin plot, where each violin represents the distribution of SHAP values for a specific feature.

In [ ]:
import shap
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train_pool)

In [ ]:
shap.summary_plot(
    shap_values,
    X,
    plot_type="bar",
    class_names=np.unique(y),
    color='purple',
    show=False
)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Mean Absolute SHAP Value', fontsize=14)
plt.title('Feature Importance by SHAP Values', fontsize=16)
plt.grid(visible=True, which='both', linestyle='--', linewidth=0.5)
plt.show()

In [ ]:
shap.summary_plot(shap_values, X.columns, plot_type="violin", show=False)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title('Feature Importance Distribution by SHAP Values', fontsize=14)
plt.grid(visible=True, which='both', linestyle='--', linewidth=0.5)
plt.show()

# Submission

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e10/sample_submission.csv')
sample_submission['loan_status'] = np.mean(test_preds, axis=0)
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)